In [16]:
import json
import os
import pickle
import cv2
import numpy as np 
import pandas as pd
from PIL import Image, ImageFile, ImageFont, ImageDraw
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
from loaddata import loaddets_md3, loaddata, load_merged, load_merged_gps

In [23]:
TRAIN_PATH = "train/"
TEST_PATH = "test/"
ANNOTATIONS_PATH = "metadata/"

size = (380,380)

ext = str(size[0]) + "x" + str(size[1])
IMAGENET_TRAIN_PATH = "./zzimagenet/train/" + ext
IMAGENET_VAL_PATH = "./zzimagenet/val/" + ext


threshold = 0.9

In [24]:
os.makedirs(IMAGENET_TRAIN_PATH)
os.makedirs(IMAGENET_VAL_PATH)

In [26]:
for f in files:
    pd.read_csv(f)

In [7]:
df_dets = loaddets()

In [8]:
df_anno, df_categories, df_images = loaddata()

In [14]:
merged = load_merged_gps()

In [15]:
merged.to_csv("train_iwildcam2021.csv")

In [9]:
def get_crop_area(bbox, image_size):
    x1, y1,w_box, h_box = bbox
    ymin,xmin,ymax, xmax = y1, x1, y1 + h_box, x1 + w_box
    area = (xmin * image_size[0], ymin * image_size[1], 
            xmax * image_size[0], ymax * image_size[1])
    return area

In [10]:
img_ids_train = []
img_idx_train = []
img_ids_test = []
img_idx_test = []

x_tot_list = []
x2_tot_list = []

In [11]:
def save_image(img, img_id, idx, is_train):  
    if is_train:
        img.save( IMAGENET_TRAIN_PATH + f"{img_id}_{idx}.jpg", format="jpeg")
        img_ids_train.append(f"{img_id}")
        img_idx_train.append(idx)
        return f"{img_id}_{idx}"
    else:
        img.save( IMAGENET_VAL_PATH + f"{img_id}_{idx}.jpg", format="jpeg")
        img_ids_test.append(f"{img_id}")
        img_idx_test.append(idx)
        return f"{img_id}_{idx}"

In [12]:
# function to calculate the averaged color and the variance
def calc_x_and_x2_tot(img):
    img = np.array(img, dtype=np.uint8)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.transpose(img,(2,0,1))
            
    return (img/255.0).reshape(-1,3).mean(0), ((img/255.0)**2).reshape(-1,3).mean(0)

In [13]:
df_train = pd.read_csv("train_split.csv")
df_valid = pd.read_csv("valid_split.csv")
df_test = pd.read_csv("test_split.csv")
df_valid.tail()

FileNotFoundError: [Errno 2] No such file or directory: 'train_split.csv'

# Main processing

In [49]:
def convert_images(annotation):
    img_id = annotation["image_id"]
    cat_id = annotation["category_id"]
    is_train = True 
    
    try:
        detections = annotation["detections"]
    except Exception as e:
        print(f"Passed {img_id}. There is no detection data.")
        return []
    path_for_train = TRAIN_PATH + img_id + ".jpg"
    path_for_test = TEST_PATH + img_id + ".jpg"
    
    if os.path.exists(path_for_train):
        file_path = path_for_train
    elif os.path.exists(path_for_test):
        file_path = path_for_test
        is_train = False
    else:
        print(f"Passed {img_id}. There is no data.")
        return []
    
    
    try:      
        img = Image.open(file_path)
    except:
        print(f"Passed {img_id}. Fail to open image.")
        print(f"pass {file_path}.")
        return (None,None)
    
    image_ids = []
    for i, detection in enumerate(detections, 1):
        if detection["conf"] < threshold:
            continue

        if detection["category"] != "1":
            continue
            
        if len(detection) == 0:
            continue
            img = img.resize(size)
            save_image(img, img_id, 0, is_train)
            
            x_tot, x2_tot = calc_mean_and_var(img)
            x_tot_list.append(x_tot)
            x2_tot_list.append(x2_tot)
        else:
            crop_area = get_crop_area(detection["bbox"], img.size)
            img_cropped = img.crop(crop_area).resize(size)
            image_id = save_image(img_cropped, img_id, i, is_train)
            image_ids.append((image_id,cat_id))
            x_tot, x2_tot = calc_x_and_x2_tot(img_cropped)
            x_tot_list.append(x_tot)
            x2_tot_list.append(x2_tot)
    return image_ids

In [3]:
df_merged = load_merged()

def save_images(df_split):
    df_split = pd.merge(df_split, df_merged, on=['category_id', 'image_id'], how='inner')
    all_images = []
    for idx,annotation in df_split.iterrows():
        image_ids = convert_images(annotation)
        if len(image_ids) == 0:
            continue
        all_images.extend(image_ids)
    df_res = pd.DataFrame(all_images, columns=["image_id","category_id"])
    return df_res

res = save_images(df_test)
res.to_csv("test_split_box.csv")

NameError: name 'df_test' is not defined

In [17]:
#image stats
img_avr =  np.array(x_tot_list).mean(0)
img_std =  np.sqrt(np.array(x2_tot_list).mean(0) - img_avr**2)
print('mean:',img_avr, ', std:', img_std)

mean: [0.37087523 0.370876   0.3708759 ] , std: [0.21022698 0.21022713 0.21022706]


In [19]:
croped_train = {"id": img_ids_train, "idx" : img_idx_train }
df_croped_train = pd.DataFrame(croped_train)
df_train_annotation = pd.DataFrame(train_annotations["annotations"])

In [20]:
df_croped_train.head()


,id,idx
0,905a3c8c-21bc-11ea-a13a-137349068a90,1
1,905a4416-21bc-11ea-a13a-137349068a90,1
2,905a4416-21bc-11ea-a13a-137349068a90,2
3,905a4416-21bc-11ea-a13a-137349068a90,3
4,905a579e-21bc-11ea-a13a-137349068a90,1


In [21]:
croped_test = {"id": img_ids_test, "idx":img_idx_test}
df_croped_test = pd.DataFrame(croped_test)

NameError: name 'df_croped_test' is not defined

In [22]:
df_croped_test.head()


,id,idx
0,915879a0-21bc-11ea-a13a-137349068a90,1
1,91588116-21bc-11ea-a13a-137349068a90,1
2,9158a2f4-21bc-11ea-a13a-137349068a90,1
3,9158aaa6-21bc-11ea-a13a-137349068a90,1
4,9158f1a0-21bc-11ea-a13a-137349068a90,1


In [23]:
df_croped_train.to_csv("./croped_train.csv", index=False)
df_croped_test.to_csv("./croped_test.csv", index=False)